In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Countries in the GapMinder Data are not exactly the same as in the UN codes.
# This function looks for the most parsimonious name link between ...
# ... the GapMinder and UN data and return the UN codes.
def compare(x, codes):
    ctry = {'size':100,'un':0}
    for i in range(len(codes)):
        if x in codes.CTRY.iloc[i]:
            if len(codes.CTRY.iloc[i]) < ctry['size']:
                ctry['size'] = len(codes.CTRY.iloc[i])
                ctry['un'] = codes.UNCTRY.iloc[i]
    return ctry

In [ ]:
countries = pd.read_csv(data_folder+'population_total.csv').country.unique()
un = []
for i in countries:
    un.append(compare(i, codes)['un'])
codes = pd.DataFrame({'CTRY':countries,'UNCTRY':un})

In [ ]:
data_folder = '../data/GapMinder_Raw_CSVs/'
codes = pd.read_csv(data_folder+'UNCTRY_CODES_GapMinderNames.csv')
codes = codes[['CTRY','UNCTRY']]

In [ ]:

data_folder = '../data/GapMinder_Raw_CSVs/'

#codes = pd.read_csv(data_folder+'UNCTRY_CODES.csv')
#codes = codes[['CTRY','UNCTRY']]

def clean_table(file,variable,start,end):
    
    df= pd.read_csv(data_folder+file)
    
    select_cols = list(map(str, np.arange(start,end,1).tolist()))
    select_cols.insert(0, "country")
    
    df=df[select_cols]
    
    df = pd.melt(df, id_vars=["country"],var_name="Date", value_name=variable)
    
    df.sort_values(by=['country','Date'])

    df = df.merge(codes, left_on='country', right_on='CTRY')
    df = df.drop("CTRY", axis=1)
    print (variable+' :')
    print ('Unique countries: ' + str(df.country.nunique()))
    print ('Unique country codes: ' + str(df.UNCTRY.nunique()))
    print ('Null Values: ' + str(df.isnull().sum().sum()))
    print('\n')
    return df

In [ ]:
start = 1980
end = 2018

#Demographic

# Mid year population
population = clean_table('population_total.csv','POPULATION',start,end)

# Crude birth rate per 1000 per year
pop_growth = clean_table('crude_birth_rate_births_per_1000_population.csv','ANNUAL_BIRTH_RATE_PER_1000',start,2016)

#Life Expectency Years
life_exp = clean_table('life_expectancy_years.csv','LIFE_EXP_YEARS',start,end)

#Life Expectency Years: Female
life_exp_f = clean_table ('life_expectancy_female.csv','LIFE_EXP_YEARS_F',start,end)

#Life Expectency Year: Male
life_exp_m = clean_table('life_expectancy_male.csv','LIFE_EXP_YEARS_M',start,end)

#Infant mortality: https://www.gapminder.org/data/documentation/gd005/
inf_mort = clean_table('child_mortality_0_5_year_olds_dying_per_1000_born.csv','CHILD_MORTALITY',start,end)



#Economic
# Total GDP: Inflation adjusted, indexed to 2010
total_gdp = clean_table('total_gdp_us_inflation_adjusted.csv','GDP_USD',start,end)

# Inflation: Annual %
inflation = clean_table('inflation_annual_percent.csv','INFLATION_PERCENT',start, end)

# % Employment for 15+ years:
employment = clean_table('aged_15plus_employment_rate_percent.csv','EMPLOYMENT',1991,end)

# GINI inequality
gini = clean_table('inequality_index_gini.csv','GINI',start,end)

# Aid recieved per person (USD indexed to 'current year', assumed to be 2017): https://data.worldbank.org/indicator/DT.ODA.ODAT.PC.ZS
aid_received = clean_table('aid_received_per_person_current_us.csv',"AID_RECEIVED_PP",start,2017)





#Education
#ratio of boys v girls in primary and secondary education: https://data.worldbank.org/indicator/SE.ENR.PRSC.FM.ZS
gender_eq_edd = clean_table('ratio_of_girls_to_boys_in_primary_and_secondary_education_perc.csv','GIRLS_V_BOYS_EDU',start,end)

#Literacy rate: % adults aged 15 and above
adult_lit_rate = clean_table('literacy_rate_adult_total_percent_of_people_ages_15_and_above.csv','ADULT_LIT_RATE',start,2012)

#Literacy rate: % Female adults aged 15 and above
adult_lit_rate_F = clean_table('literacy_rate_adult_female_percent_of_females_ages_15_above.csv','ADULT_LIT_RATE_F',start,2012)

#Literacy rate: % Male adults aged 15 and above
adult_lit_rate_M = clean_table('literacy_rate_adult_male_percent_of_males_ages_15_and_above.csv','ADULT_LIT_RATE_M',start,2012)





#SOCIETY

#Number of journalists killed
journalists_killed = clean_table('num_of_journalists_killed.csv','JOURNALISTS_KILLED',1992,end)

#Cell Phones/100 peopl
cell_phone = clean_table('cell_phones_per_100_people.csv','CELL_PHONE_PER_100',start,end)

#Corruption Perception Index : https://www.transparency.org/research/cpi
CPI = clean_table('corruption_perception_index_cpi.csv','CORRUPTION_INDEX',2012,end)

#Democracy Score:  https://www.systemicpeace.org/polityproject.html
dem_score = clean_table('democracy_score_use_as_color.csv','DEMOCRACY_SCORE',start,2012)

#Human Development Score:
human_dev_score = clean_table('hdi_human_development_index.csv', 'HUMAN_DEV_SCORE',1990,2016)

#Murders / 1000 people:
murders = clean_table('murder_per_100000_people.csv','MURDER_PER_1000',start,2017)








In [ ]:


from functools import reduce
data_frames = [population, pop_growth, life_exp,life_exp_f,life_exp_m,inf_mort,total_gdp,inflation,employment,gini,aid_received,gender_eq_edd,adult_lit_rate,adult_lit_rate_F,adult_lit_rate_M,journalists_killed,cell_phone,CPI,dem_score,human_dev_score,murders]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['UNCTRY','Date','country'],how='outer'), data_frames)


In [ ]:
df_merged.to_csv('../data/GapminderData.csv')